## 1. Importando datos sobre los precios
<p>¿Por qué algunos alientos son más caros en otras ubicaciones? En este proyecto, exploraremos series sobre los precios de los alimentos en Ruanda de las bases del <a href="https://data.humdata.org/dataset/wfp-food-prices">Programa Humanitario de la Organización de las Naciones Unidas</a>. La agricultura representa más del 30% de la economía de Ruanda y más del 60% de sus ingresos de exportación (<a href="https://www.cia.gov/library/publications/the-world-factbook/geos/rw.html">CIA Factbook</a>), así que el precio de los alimentos es muy importante para el sustento de muchos ruandeses.</p>
<p>El siguiente mapa muestra la distribución territorial de Ruanda, dividiéndola en cinco regiones administratibas: El área central alrededor de Kigali, la ciudad capítal, y las otras al Norte, Sur Este y Oeste.</p>
<p><img src="https://assets.datacamp.com/production/project_515/img/RwandaGeoProvinces.png" alt="Un mapa de las cinco regiones administrativas de Ruanda"></p>
<p>En este Jupyter Notebook vamos a importar, manipular y visualizar datos sobre los precios de la papa en Ruanda. Posteriormente, incluiremos en nuestro análisis el uso de funciones para facilitar su réplica en los precios de otros alimentos.</p>

In [ ]:
# Carga la paquetería dplyr 
....
....

# Importa los datos de "datos/papas.csv" 
precios_papas <- ....

# Utiliza glimpse() en el nuevo objeto creado
....

## 2. Dejemos los datos que usaremos
<p>Muchas de las columnas de nuestros datos en <code>precios_papa</code> no son muy útiles para nuestro análisis. Por ejemplo, la variable <code>adm0_name</code> siempre tiene el valor <code>"Rwanda"</code>, y <code>cur_name</code> tiene solo valores <code>"RWF"</code>. (Como una abreviatura en inglés del franco ruandés. Como contexto, con el tipo de cambio de junio de 2020, un peso mexicano es igual a 42 francos ruandeces.) Del mismo modo, no necesitamos para nada las columnas de ID o la fuente de los datos.</p>
<p>Incluso las columnas que necesitamos tienen nombres poco útiles. Por ejemplo, <code>adm1_name</code> no es tan claro como <code>region</code> y <code>mkt_name</code> puede ser renombrada como <code>mercado</code>. Uno de los principales errores en el análisis de datos es no entender lo que significa una variable, por lo que nombrarla claramente es una forma útil de evitar cualquier equivocación.  Un consejo útil es que cualquier variable que incluya una unidad, debe incluir dicha medida en el nombre de la variable. Aquí, los precios se dan en francos ruandeses, por lo que <code>precio_rwf</code> es un buen nombre.</p>

In [ ]:
# Selecciona las variables de interes y renombra
# las que necesiten un mejor identificador puedes
# hacerlo con dplyr::select() y dplyr::mutate()
precios_papas_2 <- ....

# Revísalo con glimpse()
....

## 3. Limpia tu base
<p>Como suele ser el caso, los datos que se nos dan no están aún en la forma que nos gustaría. Por ejemplo, en el <code>glimpse()</code> anterior podemos observar que el mes y el año se dieron como <code>int</code>. Como realizaremos algunos análisis de series de tiempo, sería útil si se proporcionaran como datos de tipo fecha. Antes de que podamos analizar los datos, necesitamos limpiarlos.</p>

In [ ]:
# Carga lubridate
....

# Convierte las variables mp_month y mp_year a fechas
precios_papas_limpio <- ....

# Revísalo con glimpse()
....

## 4. Comer solo papas no es una dieta balanceada
<p>Aunque la versatilidad de las papas es muy amplia, con su capacidad de hervirse, asarse, machacarse o freirse, la gente de Ruanda tiene gustos culinarios más variados. ¡Eso significa que tendremos que analizar algunos otros tipos de alimentos!</p>
<p>Si queremos hacer una tarea similar muchas veces, podríamos simplemente cortar y pegar nuestro código y cambiar pequeños pedazos aquí y allá. Esta es una idea terrible, ya que cambiar el código en un lugar no lo mantiene actualizado en los otros lugares, y rápidamente terminamos con muchos errores en nuestro script.</p>
<p>Una mejor idea es escribir una función. De esa forma evitamos errores de cortar y pegar y podemos tener un código más legible.</p>

In [ ]:
# Cambia el código de abajo a una función que de argumento
# utilice el nombre del archivo por cargar
# Nombra la función fx_carga_limpia

precios_papas <- read.csv("datos/papas.csv",
                          stringsAsFactors=FALSE)

precios_papas_2 <- precios_papas %>% 
  dplyr::rename(
    region = adm1_name, 
    mercado = mkt_name,
    bien = cm_name,
    mes = mp_month,
    anio = mp_year,
    precio_rwf = mp_price
  )

precios_papas_limpios <- precios_papas_2 %>% 
  mutate(
    fecha = ymd(paste(anio, mes, "01"))
  ) %>% 
  select(-mes, -anio)

# Pruébala en otro archivo
precios_chicharos <- fx_carga_limpia(...)
glimpse(precios_chicharos)

## 5. Grafica el precio de las papas
<p>Un gran primer paso en cualquier análisis de datos es observar su evolución. En este caso, tenemos algunos precios y tenemos algunas fechas, por lo que lo obvio es ver cómo esos precios cambian con el tiempo.</p>

In [ ]:
# Carga ggplot2
....

# Grafica una serie de tiempo del precio separando por
# grupo en aes(group=mercado) cada categoría 
....

## 6. Necesitamos muchos gráficos
<p>Existe una leve tendencia en los precios de la papa a aumentar hasta 2013, después de lo cual se estabilizaron. Sin embargo, lo más llamativo es la estacionalidad: los precios son más bajos alrededor de diciembre y enero, y tienen un pico alrededor de agosto. Algunos años también muestran un segundo pico alrededor de abril o mayo.</p>
<p>Al igual que con el código de importación y limpieza, si queremos hacer muchos gráficos similares, necesitamos convertir el código que escibiste en una función.</p>

In [ ]:
# Convierte este código en la función
# Nómbrala fx_grafica()
precios_papas_limpios %>% 
  ggplot(aes(fecha, precio_rwf, group = mercado)) +
  geom_line(alpha = 0.2) +
  labs(title="Evolución del precio (papas)")

# Trata de usar la gráfica en los datos de chícharos
....

## 7. Utiliza una función dentro de tu función
<p>Vamos a automatizar el proceso de graficación de cada alimento. Primero debemos crear una nueva función (<code>fx_final()</code>) que nos permita hacer ambas partes del proceso.</p>

In [ ]:
# Utiliza fx_carga_limpia() y fx_grafica en
# un nueva función
fx_final <- function(...){
    ...
}

# Aplica tu nueva función a la base "datos/yuca.csv"
fx_final("yuca.csv")

## 8. Automaticemos la graficación
<p>Utiliza las funciones de <code>purrr</code> para crear una gráfica de todas las bases que aparecen en la carpeta "datos". Recuerda que puedes obtener todos los archivos que se encuentran en dicha carpeta con <code>list.files("datos")</code>.</p>

In [ ]:
# Crea una gráfica para cada archivo en "datos"
...